In [197]:
import json
import ast
import pandas as pd
import numpy as np
import datetime

In [232]:
def get_data(n=20000):
    '''
    Gets data from a csv and cleans it
    
    '''
    
    
    df = pd.read_csv('bird_data_test.csv')
    df.drop(['code', 'captive', 'battery_level', 'location_group'], axis=1, inplace=True)
    
    #For testing so that everything will run quickly
    df = df.head(n)
    
    #Remove repeated data
    df['time_group_seconds'] =(pd.to_datetime(df['time_group']) - datetime.datetime(1970,1,1)).dt.total_seconds()
    df['date'] = pd.to_datetime(df['time_group']).dt.date
    df = add_rounded_time(df) 
    df = drop_repeated_data(df)

    #Add columns for lat, long, count, and grid_location
    
    df = add_lat_long(df)
    df['count'] = 1
    df['grid_location'] = 0

    
    #Applies a location based on a grid over Oakland to each scooter
    df = add_grid_location(df)
    og_df = df
    og_df = add_day_of_week(og_df)
    og_df = add_rounded_time(og_df)
    
    #Reforms dataframe to calculate count of scooters in each grid location every 15 min
    df = grid_count(df)
    
    #add id list and day of week
    new_df = add_id_list(df, og_df)
    new_df = add_day_of_week(new_df)
    
    new_df = add_idle_and_turnover(new_df)
    
    return new_df

In [199]:
def add_lat_long(df):
    
    df["location"] = df.location.str.replace("'", "\"").map( lambda x: json.loads(x) )
    
    df["latitude"] = df["location"].map( lambda x:x["latitude"] )
    df["longitude"] = df["location"].map( lambda x:x["longitude"] )
    
    df.drop(['location'], axis=1, inplace=True)

    df['latitude'] = df['latitude'].round(5)
    df['longitude'] = df['longitude'].round(5)
    
    return df

In [200]:
def add_rounded_time(df, interval=15):
    '''
    Adds a column with the rounded time to the interval specified.
    
    '''
    df['time_of_day'] = pd.to_datetime(df['time_group_seconds'], unit='s').dt.round('15min')  

    df_time = pd.to_datetime(df['time_of_day'])

    df['time_of_day'] = (pd.to_datetime(df['date']) - datetime.datetime(1970,1,1)).dt.total_seconds() + df_time.dt.hour*3600+df_time.dt.minute*60 + df_time.dt.second
    
    
    return df

In [201]:
def drop_repeated_data(df):
    '''
    Removes repeated data based on id and time_group_seconds - this should cut the data down by more than half
    '''
    
    df.drop_duplicates(subset=['id','time_of_day'], keep='first', inplace=True)
    return df

In [202]:
def add_grid_location(df, n=42):
    
    '''
    Assigns each data point to a location on the grid according to its lat/long
    '''
    
    top_right = [df['latitude'].max(), df['longitude'].max()]
    top_left = [df['latitude'].min(), df['longitude'].min()]
    
    grid = get_geojson_grid(top_right, top_left, n)
    
    for i, box in enumerate(grid):
        upper_right = box["properties"]["upper_right"]
        lower_left = box["properties"]["lower_left"]
    
        mask = (
            (df.latitude <= upper_right[1]) & (df.latitude >= lower_left[1]) &
            (df.longitude <= upper_right[0]) & (df.longitude >= lower_left[0])
           )
    
        column_name = 'grid_location'
        df.loc[mask, column_name] = i
    
    return df

In [203]:
def get_geojson_grid(upper_right, lower_left, n=6):
    """Returns a grid of geojson rectangles, and computes the exposure in each section of the grid based on the vessel data.

    Parameters
    ----------
    upper_right: array_like
        The upper right hand corner of "grid of grids" (the default is the upper right hand [lat, lon] of the USA).

    lower_left: array_like
        The lower left hand corner of "grid of grids"  (the default is the lower left hand [lat, lon] of the USA).

    n: integer
        The number of rows/columns in the (n,n) grid.

    Returns
    -------

    list
        List of "geojson style" dictionary objects   
    """

    all_boxes = []

    lat_steps = np.linspace(lower_left[0], upper_right[0], n+1)
    lon_steps = np.linspace(lower_left[1], upper_right[1], n+1)

    lat_stride = lat_steps[1] - lat_steps[0]
    lon_stride = lon_steps[1] - lon_steps[0]

    for lat in lat_steps[:-1]:
        for lon in lon_steps[:-1]:
            # Define dimensions of box in grid
            upper_left = [lon, lat + lat_stride]
            upper_right = [lon + lon_stride, lat + lat_stride]
            lower_right = [lon + lon_stride, lat]
            lower_left = [lon, lat]

            # Define json coordinates for polygon
            coordinates = [
                upper_left,
                upper_right,
                lower_right,
                lower_left,
                upper_left
            ]

            geo_json = {"type": "FeatureCollection",
                        "properties":{
                            "lower_left": lower_left,
                            "upper_right": upper_right
                        },
                        "features":[]}

            grid_feature = {
                "type":"Feature",
                "geometry":{
                    "type":"Polygon",
                    "coordinates": [coordinates],
                }
            }

            geo_json["features"].append(grid_feature)

            all_boxes.append(geo_json)

    return all_boxes

In [204]:
def grid_count(df, n=42):
    '''
    For a given time, day of the week create df of number of scooters in each grid location
    '''
    
    #COMBINE UNIQUE DATE AND UNIQUE ROUNDED TIME INTO 1 LOOP INSTEAD OF 2
    
    new_df = pd.DataFrame()
    for unique_rounded_time in df['time_of_day'].unique():
        for unique_grid_location in range(n**2):
            if ((df['time_of_day'] == unique_rounded_time) & (df['grid_location'] == unique_grid_location)).any():
                continue
            else:
                new_df = new_df.append({'time_of_day':unique_rounded_time, 'grid_location':unique_grid_location}, ignore_index=True)
    new_df['grid_location'] = new_df['grid_location'].astype(int)                
    
    df = df.groupby(['time_of_day', 'grid_location']).size().reset_index(name='counts')
    df3 = pd.concat([df,new_df], sort=False)
    df3.fillna(value=0, inplace=True)
    df3.sort_values(by=['grid_location', 'time_of_day'], inplace=True)
    #for a given date and rounded time - check to see if there is a grid location, if not set count to 0.
    
    #df = df.groupby(['grid_location']).agg(['count'])
    return df3

In [231]:
def add_day_of_week(df):
    
    df['date'] = pd.to_datetime(df['time_of_day'], unit='s').dt.date
    
    #df['date'] = pd.to_datetime(df['time_of_day'], unit='s').dt.round("D")
    df['day_of_week'] = pd.to_datetime(df['date']).dt.day_name()
    
    
    return df

In [206]:
def add_id_list(df, og_df):

    g = og_df.groupby(['grid_location', 'time_of_day'])['id'].apply(list).reset_index(name='id_list')

    merger_df = pd.merge(df, g, on=['grid_location', 'time_of_day'], how='outer')
    
    isnull = merger_df.id_list.isnull()

    merger_df.loc[isnull, 'id_list'] = [ [[]] * isnull.sum() ]
    
    return merger_df

In [243]:
def add_idle_and_turnover(merger_df):
    
    group1_master = pd.DataFrame()
    group2_master = pd.DataFrame()
    for grid_location in merger_df['grid_location'].unique():
        
        group1 = merger_df[merger_df['grid_location']==grid_location].iloc[:-1]
        group1.reset_index(inplace=True)
        group1_master = group1_master.append(group1, ignore_index=True)
        
        group2 = merger_df[merger_df['grid_location']==grid_location].iloc[1:]
        group2.reset_index(inplace=True)
        group2_master = group2_master.append(group2, ignore_index=True)

    df_id_list = group1_master.merge(group2_master, how='outer', left_index=True, right_index=True)
    
    df_id_list['idle'] = df_id_list.apply(lambda x: [i for i in x['id_list_x'] if i.lower() in x['id_list_y']], axis=1)
    
    df_id_list['turn_over'] = df_id_list.apply(lambda x: [i for i in x['id_list_y'] if i.lower() not in x['id_list_x']], axis=1)
    
    df_id_list['num_idle_15min'] = df_id_list['idle'].str.len()
    df_id_list['num_turn_over_15min'] = df_id_list['turn_over'].str.len()
    
    
    df_id_list['time'] = df_id_list['time_of_day_y'] - (pd.to_datetime(df_id_list['date_y']) - datetime.datetime(1970,1,1)).dt.total_seconds()
    
    #df_id_list.drop(['index_x','time_of_day_x','grid_location_x','counts_x', 'id_list_x', 'date_x', 'day_of_week_x'], axis=1, inplace=True)
    
    return df_id_list

In [244]:
df = get_data()

In [246]:
df

,index_y,time_of_day_y,grid_location_y,counts_y,id_list_y,date_y,day_of_week_y,idle,turn_over,num_idle_15min,num_turn_over_15min,time
0,1,1.542050e+09,0,0.0,[],2018-11-12,Monday,[],[],0,0,69300.0
1,3,1.542050e+09,1,0.0,[],2018-11-12,Monday,[],[],0,0,69300.0
2,5,1.542050e+09,2,0.0,[],2018-11-12,Monday,[],[],0,0,69300.0
3,7,1.542050e+09,3,0.0,[],2018-11-12,Monday,[],[],0,0,69300.0
4,9,1.542050e+09,4,0.0,[],2018-11-12,Monday,[],[],0,0,69300.0
5,11,1.542050e+09,5,0.0,[],2018-11-12,Monday,[],[],0,0,69300.0
6,13,1.542050e+09,6,0.0,[],2018-11-12,Monday,[],[],0,0,69300.0
7,15,1.542050e+09,7,0.0,[],2018-11-12,Monday,[],[],0,0,69300.0
8,17,1.542050e+09,8,0.0,[],2018-11-12,Monday,[],[],0,0,69300.0
9,19,1.542050e+09,9,0.0,[],2018-11-12,Monday,[],[],0,0,69300.0


In [237]:
df['time_of_day_y'] - (pd.to_datetime(df['date_y']) - datetime.datetime(1970,1,1)).dt.total_seconds()

0       69300.0
1       69300.0
2       69300.0
3       69300.0
4       69300.0
5       69300.0
6       69300.0
7       69300.0
8       69300.0
9       69300.0
10      69300.0
11      69300.0
12      69300.0
13      69300.0
14      69300.0
15      69300.0
16      69300.0
17      69300.0
18      69300.0
19      69300.0
20      69300.0
21      69300.0
22      69300.0
23      69300.0
24      69300.0
25      69300.0
26      69300.0
27      69300.0
28      69300.0
29      69300.0
         ...   
1734    69300.0
1735    69300.0
1736    69300.0
1737    69300.0
1738    69300.0
1739    69300.0
1740    69300.0
1741    69300.0
1742    69300.0
1743    69300.0
1744    69300.0
1745    69300.0
1746    69300.0
1747    69300.0
1748    69300.0
1749    69300.0
1750    69300.0
1751    69300.0
1752    69300.0
1753    69300.0
1754    69300.0
1755    69300.0
1756    69300.0
1757    69300.0
1758    69300.0
1759    69300.0
1760    69300.0
1761    69300.0
1762    69300.0
1763    69300.0
Length: 1764, dtype: flo

In [220]:
pd.to_datetime(df['time_of_day_y'], unit ='s')

0      2018-11-13 19:15:00
1      2018-11-13 19:15:00
2      2018-11-13 19:15:00
3      2018-11-13 19:15:00
4      2018-11-13 19:15:00
5      2018-11-13 19:15:00
6      2018-11-13 19:15:00
7      2018-11-13 19:15:00
8      2018-11-13 19:15:00
9      2018-11-13 19:15:00
10     2018-11-13 19:15:00
11     2018-11-13 19:15:00
12     2018-11-13 19:15:00
13     2018-11-13 19:15:00
14     2018-11-13 19:15:00
15     2018-11-13 19:15:00
16     2018-11-13 19:15:00
17     2018-11-13 19:15:00
18     2018-11-13 19:15:00
19     2018-11-13 19:15:00
20     2018-11-13 19:15:00
21     2018-11-13 19:15:00
22     2018-11-13 19:15:00
23     2018-11-13 19:15:00
24     2018-11-13 19:15:00
25     2018-11-13 19:15:00
26     2018-11-13 19:15:00
27     2018-11-13 19:15:00
28     2018-11-13 19:15:00
29     2018-11-13 19:15:00
               ...        
1734   2018-11-13 19:15:00
1735   2018-11-13 19:15:00
1736   2018-11-13 19:15:00
1737   2018-11-13 19:15:00
1738   2018-11-13 19:15:00
1739   2018-11-13 19:15:00
1